In [1]:
import os
import json
import cv2
import numpy as np
from math import atan2, degrees
from mediapipe import solutions

In [2]:
def calculate_angle(point1, point2, point3):
    """
    Calculate the angle between three points.
    point1, point2, and point3 are coordinates in the form [x, y].
    """
    vector1 = np.array(point1) - np.array(point2)
    vector2 = np.array(point3) - np.array(point2)
    angle = atan2(vector2[1], vector2[0]) - atan2(vector1[1], vector1[0])
    angle = abs(degrees(angle))
    if angle > 180:
        angle = 360 - angle
    return angle

In [3]:
def process_image(image_path, pose_detector):
    """
    Process a single image to detect keypoints and compute joint angles.
    Returns a dictionary of calculated joint angles or None if no landmarks are detected.
    """
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose_detector.process(image_rgb)
    
    if result.pose_landmarks:
        landmarks = result.pose_landmarks.landmark
        keypoints = {
            "left_shoulder": [landmarks[11].x, landmarks[11].y],
            "right_shoulder": [landmarks[12].x, landmarks[12].y],
            "left_elbow": [landmarks[13].x, landmarks[13].y],
            "right_elbow": [landmarks[14].x, landmarks[14].y],
            "left_wrist": [landmarks[15].x, landmarks[15].y],
            "right_wrist": [landmarks[16].x, landmarks[16].y],
            "left_hip": [landmarks[23].x, landmarks[23].y],
            "right_hip": [landmarks[24].x, landmarks[24].y],
            "left_knee": [landmarks[25].x, landmarks[25].y],
            "right_knee": [landmarks[26].x, landmarks[26].y],
            "left_ankle": [landmarks[27].x, landmarks[27].y],
            "right_ankle": [landmarks[28].x, landmarks[28].y],
        }
        
        angles = {
            "armpit_left": calculate_angle(keypoints["left_elbow"], keypoints["left_shoulder"], keypoints["left_hip"]),
            "armpit_right": calculate_angle(keypoints["right_elbow"], keypoints["right_shoulder"], keypoints["right_hip"]),
            "elbow_left": calculate_angle(keypoints["left_wrist"], keypoints["left_elbow"], keypoints["left_shoulder"]),
            "elbow_right": calculate_angle(keypoints["right_wrist"], keypoints["right_elbow"], keypoints["right_shoulder"]),
            "hip_left": calculate_angle(keypoints["left_shoulder"], keypoints["left_hip"], keypoints["left_knee"]),
            "hip_right": calculate_angle(keypoints["right_shoulder"], keypoints["right_hip"], keypoints["right_knee"]),
            "knee_left": calculate_angle(keypoints["left_hip"], keypoints["left_knee"], keypoints["left_ankle"]),
            "knee_right": calculate_angle(keypoints["right_hip"], keypoints["right_knee"], keypoints["right_ankle"]),
            "ankle_left": calculate_angle(keypoints["left_knee"], keypoints["left_ankle"], [keypoints["left_ankle"][0], keypoints["left_ankle"][1] + 1]),
            "ankle_right": calculate_angle(keypoints["right_knee"], keypoints["right_ankle"], [keypoints["right_ankle"][0], keypoints["right_ankle"][1] + 1]),
        }
        return angles
    return None

In [4]:
def generate_reference_data(dataset_path, output_file):
    """
    Generate reference pose data JSON file by calculating average joint angles
    for each asana in the dataset.
    """
    import mediapipe as mp

    mp_pose = mp.solutions.pose
    reference_data = {}
    
    with mp_pose.Pose(static_image_mode=True) as pose_detector:
        for split in ["TRAIN", "TEST"]:
            split_path = os.path.join(dataset_path, split)
            asanas = os.listdir(split_path)

            for asana in asanas:
                if asana == ".DS_Store":
                    continue
                asana_path = os.path.join(split_path, asana)
                images = [os.path.join(asana_path, img) for img in os.listdir(asana_path) if img.endswith(('.png', '.jpg'))]

                angle_sums = {}
                angle_counts = {}
                
                for image_path in images:
                    angles = process_image(image_path, pose_detector)
                    if angles:
                        for joint, angle in angles.items():
                            if joint not in angle_sums:
                                angle_sums[joint] = 0
                                angle_counts[joint] = 0
                            angle_sums[joint] += angle
                            angle_counts[joint] += 1
                
                # Calculate averages for this asana
                reference_data[asana] = {
                    joint: angle_sums[joint] / angle_counts[joint]
                    for joint in angle_sums
                }
    
    # Write to JSON file
    with open(output_file, 'w') as f:
        json.dump(reference_data, f, indent=4)
    print(f"Reference data saved to {output_file}")

In [ ]:
"""
    Saving the JSON file with reference pose data
"""

dataset_path = "DATASET" 
output_file = "reference_pose_data.json"
generate_reference_data(dataset_path, output_file)